## cfos_batch_combination_20201125_qc_histograms
The purpose of this notebook is to make the quality control histograms to browse and identify outliers for further inspection and possibly removal from the dataset.

In [13]:
import pandas as pd
import json
import numpy as np, os
import matplotlib.pyplot as plt
import edt
import seaborn as sns
import graphviz


## Data loading and validation

In [14]:
main_file = '../data/animals_eroded_counts_20201125.csv'
main_file_withoutliers = '../data/animals_eroded_counts_20201125_withoutliers.csv'
pma_segment_props_file = '../data/pma_segment_properties_info'
pma_ontology_json_file = '../data/PMA_ontology.json'
brain_divisions_file = '../data/developmentalregions.xlsx' # contains the telencephalon, mesencephalon, etc regions

In [11]:
# Read in the new dataframe containing cell counts in each PMA brain region
main_df = pd.read_csv(main_file_withoutliers)
main_df

,brain,brain_longname,condition,batch,root,Midbrain,Oculomotor nucleus,Medial terminal nucleus of the accessory optic tract,Lateral terminal nucleus of the accessory optic tract,Dorsal terminal nucleus of the accessory optic tract,...,supraoptic commissures,fasciculus retroflexus,habenular commissure,stria medullaris,nigrostriatal tract,rubrospinal tract,ventral tegmental decussation,crossed tectospinal pathway,direct tectospinal pathway,doral tegmental decussation
0,an011,an011,acquisition_day1,202010_cfos,0,13350,0,34,29,24,...,22,57,0,90,95,508,41,27,0,3
1,an012,an012,acquisition_day1,202010_cfos,0,11361,22,40,14,93,...,120,64,0,79,183,387,36,40,0,4
2,an013,an013,acquisition_day1,202010_cfos,1,12563,11,68,39,5,...,26,26,1,68,71,352,26,29,0,1
3,an014,an014,acquisition_day1,202010_cfos,2,12554,3,28,62,98,...,53,70,0,83,109,197,18,51,0,0
4,an015,an015,acquisition_day1,202010_cfos,2,11431,2,20,8,4,...,38,20,1,73,53,421,23,50,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,an30_crus1_lat,an30_crus1_lat,Vehicle_control_reversal,202002_cfos,3,32942,2,216,98,427,...,88,28,3,47,122,864,92,67,0,0
167,an31_crus1_lat,an31_crus1_lat,Vehicle_control_reversal,202002_cfos,1,33377,9,108,162,324,...,85,14,0,97,110,1249,188,96,0,9
168,an32_crus1_lat,an32_crus1_lat,Vehicle_control_reversal,202002_cfos,7,36458,2,208,167,199,...,114,25,4,249,224,1657,94,157,0,4
169,an33_crus1_lat,an33_crus1_lat,Vehicle_control_reversal,202002_cfos,4,29123,4,59,128,481,...,94,39,3,29,90,978,56,107,0,2


In [19]:
for ii,brain_series in main_df.iterrows():
    brain = brain_series['brain']
    condition = brain_series['condition']
    batch = brain_series['batch']
    savename = f'../figs/qc_figs/{batch}_{condition}_{brain}_qcbarplot.png'
    if brain == 'dadult_pc_crus1_10': 
        counts = brain_series.iloc[4:].values
        print(savename,counts)

../figs/qc_figs/201810_adultacutePC_ymaze_cfos_crusI_bilateral_reversal_dadult_pc_crus1_10_qcbarplot.png [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [20]:
plt.ioff()
for ii,brain_series in main_df.iterrows():
    brain = brain_series['brain']
    condition = brain_series['condition']
    batch = brain_series['batch']
    savename = f'../figs/qc_figs/{batch}_{condition}_{brain}_qcbarplot.png'
    if os.path.exists(savename):
        continue
   
    print(savename)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    counts = brain_series.iloc[4:].values
    # names = test_brain_series.columns[4:]
    ax.bar(list(range(len(counts))),counts)
    ax.set_ylabel('Total counts in region')
    ax.set_xlabel('Region index in dataframe')
    plt.savefig(savename)

../figs/qc_figs/202010_cfos_acquisition_day1_an011_qcbarplot.png


/home/ahoag/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


../figs/qc_figs/202010_cfos_acquisition_day1_an012_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an013_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an014_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an015_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an016_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an017_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an018_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an019_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day1_an020_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day2_an021_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day2_an022_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day2_an023_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day2_an024_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day2_an025_qcbarplot.png
../figs/qc_figs/202010_cfos_acquisition_day2_an026_qcbarplot.png
../figs/qc_figs/202010_cf

../figs/qc_figs/202002_cfos_dilutedvector_control_reversal_an9_mcherrydymaze_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an001_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an002_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an003_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an004_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an005_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an006_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an007_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an008_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an009_qcbarplot.png
../figs/qc_figs/202010_cfos_habituation_an010_qcbarplot.png
../figs/qc_figs/201904_ymaze_cfos_homecage_control_an29_qcbarplot.png
../figs/qc_figs/201904_ymaze_cfos_homecage_control_an30_qcbarplot.png
../figs/qc_figs/201904_ymaze_cfos_homecage_control_an31_qcbarplot.png
../figs/qc_figs/201904_ymaze_cfos_homecage_control_an32_qcbarplot.png
../figs/qc_figs/201904_ymaze_

After looking at the plots there is a region that is very variable between 475 and 500. Let's print out those region names

In [22]:
all_region_names=main_df.columns[4:]
all_region_names[475:500]

Index(['Accessory olfactory bulb, mitral layer', 'Anterior olfactory nucleus',
       'Main olfactory bulb', 'Postpiriform transition area', 'Taenia tecta',
       'Taenia tecta, dorsal part', 'Taenia tecta, ventral part',
       'Nucleus of the lateral olfactory tract, molecular layer',
       'Nucleus of the lateral olfactory tract, pyramidal layer',
       'Nucleus of the lateral olfactory tract, layer 3',
       'Cortical amygdalar area, anterior part',
       'Cortical amygdalar area, posterior part',
       'Cortical amygdalar area, posterior part, lateral zone',
       'Cortical amygdalar area, posterior part, medial zone',
       'Piriform-amygdalar area', 'Dorsal peduncular area', 'Piriform area',
       'Hippocampal formation', 'Subiculum', 'Parasubiculum',
       'Entorhinal area, lateral part',
       'Entorhinal area, lateral part, layer 2',
       'Entorhinal area, lateral part, layer 6a',
       'Entorhinal area, lateral part, layer 3',
       'Entorhinal area, lateral p

In [32]:
for ii,brain_series in main_df.iterrows():

        brain = brain_series['brain']
        condition = brain_series['condition']
        batch = brain_series['batch']
        if batch == '201904_ymaze_cfos' and brain == 'an8':
            savename = f'../figs/qc_figs/{batch}_{condition}_{brain}_qcbarplot.png' 
            counts = brain_series.iloc[4:].values
            suspect_counts = counts[475:500]
            print(batch,brain,brain_series.iloc[480:500])
            print()

201904_ymaze_cfos an8 Anterior olfactory nucleus                                   957
Main olfactory bulb                                          681
Postpiriform transition area                                1035
Taenia tecta                                                   6
Taenia tecta, dorsal part                                    590
Taenia tecta, ventral part                                   197
Nucleus of the lateral olfactory tract, molecular layer      135
Nucleus of the lateral olfactory tract, pyramidal layer      119
Nucleus of the lateral olfactory tract, layer 3               36
Cortical amygdalar area, anterior part                       685
Cortical amygdalar area, posterior part                        9
Cortical amygdalar area, posterior part, lateral zone       1091
Cortical amygdalar area, posterior part, medial zone        1262
Piriform-amygdalar area                                     1285
Dorsal peduncular area                                       443
Pir